# Elastic engine demo

In [1]:
import datalabframework as dlf
dlf.project.info()

{'workrun': 'default',
 'filename': 'elastic.ipynb',
 'rootpath': '/home/thuc/teko/product-search/dlf-demos_fork/demos/elastic',
 'workdir': '/home/thuc/teko/product-search/dlf-demos_fork/demos/elastic'}

In [2]:
metadata = dlf.params.metadata()
dlf.utils.pretty_print(metadata)
# just the resources, some pretty printing
dlf.utils.pretty_print(metadata['resources'])

engines:
  spark-local:
    config:
      jobname: default
      master: local[*]
    context: spark
loggers:
  stream:
    enable: true
    severity: info
providers:
  elastic_test:
    hostname: 123.31.32.226
    port: 9200
    service: elastic
  local_filesystem:
    format: csv
    path: data
    service: local
resources:
  .keywords:
    path: keywords.csv
    provider: local_filesystem
    read:
      mapping:
        abc:
          drop: true
          fields:
            keyword:
              ignore_above: 256
              type: keyword
          fill: value
          name: cds
          type: text
      options:
        header: true
        inferSchema: true
  .keywords_elk:
    index: search_keywords_dev2
    mappings:
      doc_type: keyword
      properties:
        api_result: integer
        count: integer
        gg: integer
        gg_clicks: double
        gg_impressions: double
        gg_position: double
        pv: integer
        pv_impressions: double
        pv

In [1]:
import datalabframework as dlf
engine = dlf.engines.get('spark-local')

PYSPARK_SUBMIT_ARGS:  pyspark-shell


In [2]:
# dlf.data.metadata('keywords_elk')

In [10]:
res = engine.read(".keywords_search", _query="laptop i7", _from=0, _size=100)
res.show()

repartition  None
coalesce  None
cache False
{'size': 100, 'from': 0, 'query': {'function_score': {'query': {'match': {'query_wo_tones': {'query': 'laptop i7', 'operator': 'or', 'fuzziness': 'AUTO'}}}, 'script_score': {'script': {'source': "Math.sqrt(doc['impressions'].value)"}}, 'boost_mode': 'multiply'}}}
Summary: {'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'total_hits': 212, 'max_score': 645.10345}
+--------------------+--------------------+---------+-------+----------+-----+---+--------------+------------------+-----------+--------+---+---------+----------------+----------------+
|                 _id|              _index|   _score|  _type|api_result|count| gg|gg_impressions|       gg_position|impressions|position| pv|pv_result|           query|  query_wo_tones|
+--------------------+--------------------+---------+-------+----------+-----+---+--------------+------------------+-----------+--------+---+---------+---------------

In [4]:
type(res)

pyspark.sql.dataframe.DataFrame

In [5]:
df = engine.read(".keywords")

repartition  None
coalesce  None
cache False
file:///home/thuc/teko/product-search/dlf-demos_fork/demos/elastic/data/keywords.csv


In [9]:
df.show()

+--------------------+--------------------+-----------+--------+--------------+-----------+----------+---------+---+---+-----+
|      query_wo_tones|               query|impressions|position|gg_impressions|gg_position|api_result|pv_result| pv| gg|count|
+--------------------+--------------------+-----------+--------+--------------+-----------+----------+---------+---+---+-----+
|        asus h110m-d|        asus h110m-d|195.8090665|    null|         570.0|3.335087719|     458.0|      0.0|  0|  1|    1|
|           ddr3 1600|           ddr3 1600|      202.0|     1.0|          86.0|        2.5|      30.0|     36.0|  1|  1|    1|
|         epson l1300|         epson l1300|191.3432457|    null|         557.0|5.057450628|       1.0|      1.0|  0|  1|    1|
| intel pentium g4600| intel pentium g4600|166.2659442|    null|         484.0|3.423553719|       1.0|      0.0|  0|  1|    1|
|               j250f|               j250f|220.1993186|    null|         641.0|3.653666147|       1.0|      3.0

In [6]:
engine.write(df, ".keywords_elk", mode='overwrite')

repartition  None
coalesce  None
cache False
{'index': {'mapping': {'total_fields': {'limit': 1024}}, 'number_of_replicas': 3, 'number_of_shards': 1}}
{'api_result': {'type': 'integer'}, 'count': {'type': 'integer'}, 'gg': {'type': 'integer'}, 'gg_clicks': {'type': 'double'}, 'gg_impressions': {'type': 'double'}, 'gg_position': {'type': 'double'}, 'pv': {'type': 'integer'}, 'pv_impressions': {'type': 'double'}, 'pv_position': {'type': 'double'}, 'pv_result': {'type': 'double'}, 'query': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}}, 'type': 'text'}, 'query_wo_tones': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}}, 'type': 'text'}}
